# This notebook will present a project focusing on Credit Risk.
## Plan

Predict the probability of default by various ML models:
- Logistic regression
- Discriminant Analysis
- Naive Bayesian
- KNN
- RF
- Classification trees
- Neural networks

## Data

### Data Source: UCI Machine Learning Repository.

- https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients#
- https://bradzzz.gitbooks.io/ga-seattle-dsi/content/dsi/dsi_05_classification_databases/2.1-lesson/assets/datasets/DefaultCreditCardClients_yeh_2009.pdf

### Data Set Information:

This research aimed at the case of customers default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods. From the perspective of risk management, the result of predictive accuracy of the estimated probability of default will be more valuable than the binary result of classification - credible or not credible clients. Because the real probability of default is unknown, this study presented the novel Sorting Smoothing Method to estimate the real probability of default. With the real probability of default as the response variable (Y), and the predictive probability of default as the independent variable (X), the simple linear regression result (Y = A + BX) shows that the forecasting model produced by artificial neural network has the highest coefficient of determination; its regression intercept (A) is close to zero, and regression coefficient (B) to one. Therefore, among the six data mining techniques, artificial neural network is the only one that can accurately estimate the real probability of default.

### Attribute Information:

This research employed a binary variable, default payment (Yes = 1, No = 0), as the response variable. 

This study reviewed the literature and used the following 23 variables as explanatory variables:

- X1: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit.
- X2: Gender (1 = male; 2 = female).
- X3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others).
- X4: Marital status (1 = married; 2 = single; 3 = others).
- X5: Age (year).
- X6 - X11: History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: 

    - X6 = the repayment status in September, 2005; 
    - X7 = the repayment status in August, 2005; 
    - . . .;
    - X11 = the repayment status in April, 2005. 

> The measurement scale for the repayment status is: 
- -1 = pay duly; 
- 1 = payment delay for one month; 
- 2 = payment delay for two months; 
- . . .; 
- 8 = payment delay for eight months; 
- 9 = payment delay for nine months and above.


- X12-X17: Amount of bill statement (NT dollar). 
    - X12 = amount of bill statement in September, 2005; 
    - X13 = amount of bill statement in August, 2005; 
    - . . .; 
    - X17 = amount of bill statement in April, 2005.

- X18-X23: Amount of previous payment (NT dollar). 
    - X18 = amount paid in September, 2005; 
    - X19 = amount paid in August, 2005; 
    - . . .;
    - X23 = amount paid in April, 2005.

## Environment Setup and EDA

In [15]:
import pandas as pd
import numpy as np 

import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline 
import warnings
warnings.filterwarnings('ignore')

In [17]:
data = pd.read_excel('Credit_Risk_Prediction\data_uci\default_of_credit_card_clients.xls', header=[0,1], index_col=0)
data.columns = data.columns.map(':'.join)
data.head()

,X1:LIMIT_BAL,X2:SEX,X3:EDUCATION,X4:MARRIAGE,X5:AGE,X6:PAY_0,X7:PAY_2,X8:PAY_3,X9:PAY_4,X10:PAY_5,...,X15:BILL_AMT4,X16:BILL_AMT5,X17:BILL_AMT6,X18:PAY_AMT1,X19:PAY_AMT2,X20:PAY_AMT3,X21:PAY_AMT4,X22:PAY_AMT5,X23:PAY_AMT6,Y:default payment next month
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 1 to 30000
Data columns (total 24 columns):
X1:LIMIT_BAL                    30000 non-null int64
X2:SEX                          30000 non-null int64
X3:EDUCATION                    30000 non-null int64
X4:MARRIAGE                     30000 non-null int64
X5:AGE                          30000 non-null int64
X6:PAY_0                        30000 non-null int64
X7:PAY_2                        30000 non-null int64
X8:PAY_3                        30000 non-null int64
X9:PAY_4                        30000 non-null int64
X10:PAY_5                       30000 non-null int64
X11:PAY_6                       30000 non-null int64
X12:BILL_AMT1                   30000 non-null int64
X13:BILL_AMT2                   30000 non-null int64
X14:BILL_AMT3                   30000 non-null int64
X15:BILL_AMT4                   30000 non-null int64
X16:BILL_AMT5                   30000 non-null int64
X17:BILL_AMT6                   30000 non-n

In [18]:
data.describe()

,X1:LIMIT_BAL,X2:SEX,X3:EDUCATION,X4:MARRIAGE,X5:AGE,X6:PAY_0,X7:PAY_2,X8:PAY_3,X9:PAY_4,X10:PAY_5,...,X15:BILL_AMT4,X16:BILL_AMT5,X17:BILL_AMT6,X18:PAY_AMT1,X19:PAY_AMT2,X20:PAY_AMT3,X21:PAY_AMT4,X22:PAY_AMT5,X23:PAY_AMT6,Y:default payment next month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,-0.266200,...,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,1.133187,...,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


In [14]:
data.isnull()

,X1:LIMIT_BAL,X2:SEX,X3:EDUCATION,X4:MARRIAGE,X5:AGE,X6:PAY_0,X7:PAY_2,X8:PAY_3,X9:PAY_4,X10:PAY_5,...,X15:BILL_AMT4,X16:BILL_AMT5,X17:BILL_AMT6,X18:PAY_AMT1,X19:PAY_AMT2,X20:PAY_AMT3,X21:PAY_AMT4,X22:PAY_AMT5,X23:PAY_AMT6,Y:default payment next month
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
29997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
29998,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
29999,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
